# Initial setup

In [2]:
#importing packages needed for analysis
import os
import numpy as np
import pandas as pd
import math
from pandas import DataFrame

path = os.getcwd()
parent = os.path.dirname(path)
print(path)
print(parent)
#os.mkdir(path+'\outputs')

C:\Users\cmarcy\Desktop\py_projects\plantfile\NEEDS_project
C:\Users\cmarcy\Desktop\py_projects\plantfile


# Reviewing GDX files

In [3]:
from os import listdir

def find_filenames( path_to_dir, suffix=".xlsx" ):
    filenames = listdir( path_to_dir )
    return [ filename for filename in filenames if filename.endswith( suffix ) ]
filenames = find_filenames('background/gdx_files')
print(filenames)

name_list = []
import xlrd
for name in filenames:
    sheets = xlrd.open_workbook(('background/gdx_files/'+name), on_demand=True)
    data = [name,sheets.sheet_names()]
    name_list.append(data)

name_list = pd.DataFrame(name_list, columns = ['names','sheets'])
print(name_list)

['existing_units.xlsx', 'prescriptive_builds.xlsx', 'prescriptive_retires.xlsx']
                       names  \
0        existing_units.xlsx   
1   prescriptive_builds.xlsx   
2  prescriptive_retires.xlsx   

                                              sheets  
0                [CONVOLDqctn, import_store, others]  
1                       [nonqn, nqct, storage, wind]  
2  [nuke60retire, nuke80retire, nukeearlyretire, ...  


In [4]:
#turns the column of sheet lists into separate rows matched to each file name
nl = pd.DataFrame([[x] + [z] for x, y in name_list.values for z in y], \
                  columns=name_list.columns)
print(nl)
#Note: can't get explode to work, so found this alternative (above) on the internet.  
#print(name_list.explode('sheets'))

                        names               sheets
0         existing_units.xlsx          CONVOLDqctn
1         existing_units.xlsx         import_store
2         existing_units.xlsx               others
3    prescriptive_builds.xlsx                nonqn
4    prescriptive_builds.xlsx                 nqct
5    prescriptive_builds.xlsx              storage
6    prescriptive_builds.xlsx                 wind
7   prescriptive_retires.xlsx         nuke60retire
8   prescriptive_retires.xlsx         nuke80retire
9   prescriptive_retires.xlsx      nukeearlyretire
10  prescriptive_retires.xlsx        nukerefretire
11  prescriptive_retires.xlsx       precriptiveret
12  prescriptive_retires.xlsx   winretireexisiting
13  prescriptive_retires.xlsx  windretireprescribe


In [5]:
#test = pd.read_excel('background/gdx_files/' + nl['names'][0], sheetname = nl['sheets'][0])

gdx_tables = {}
for x in nl.index:
    gdx_tables[nl['sheets'][x]] = \
    pd.read_excel('background/gdx_files/' + nl['names'][x], sheet_name = nl['sheets'][x], header=None)
print(gdx_tables)

{'CONVOLDqctn':               0     1       2         3
0    CoalOldScr   p10  recirc   101.200
1    CoalOldScr  p101  recirc  3033.000
2    CoalOldScr  p101    once  3045.000
3    CoalOldScr  p103  recirc  1641.000
4    CoalOldScr  p103    once   903.400
5    CoalOldScr  p105  recirc  1194.000
6    CoalOldScr  p107  recirc  4147.200
7    CoalOldScr  p107    once   508.000
8    CoalOldScr  p107    pond  1596.000
9    CoalOldScr  p108  recirc  1024.000
10   CoalOldScr  p108    once   137.000
11   CoalOldScr  p109  recirc  3052.000
12   CoalOldScr  p111    once   216.000
13   CoalOldScr  p112  recirc  3430.000
14   CoalOldScr  p112    once  1560.000
15   CoalOldScr  p114  recirc  1323.400
16   CoalOldScr  p115  recirc  2160.000
17   CoalOldScr  p116  recirc  3874.000
18   CoalOldScr  p116    once    50.000
19   CoalOldScr  p116    pond    74.000
20   CoalOldScr   p12  recirc   268.000
21   CoalOldScr  p122  recirc  2560.400
22   CoalOldScr  p123  recirc  1273.000
23   CoalOldScr  p123   

In [5]:
for key in gdx_tables.keys():
    if key != 'others':
        print('\033[1m',key,'\033[0m')
        key = pd.DataFrame.from_dict(gdx_tables[key])
        print('original table layout:')
        print(key.head(3))
        print()

        temp = key.copy()

        ss = str(temp.iloc[0,0])
        if ss.isnumeric(): 
            temp.drop(temp.columns[0], axis =1, inplace=True) 
            #print(temp.head(2))

        ss = str(temp.iloc[0,0])
        if ss.isnumeric(): 
            temp.drop(temp.columns[0], axis =1, inplace=True) 
            #print(temp.head(2))

        ss = str(temp.iloc[0,0])
        if ss[0:1] == 'p': 
            #print(ss[0:2])
            temp.drop(temp.columns[0], axis =1, inplace=True) 
            #print(temp.head(2))

        #print(temp.head(2))
        #print()
        temp2=temp.groupby([temp.columns[0]],as_index=False)[temp.columns[-1]].sum()
        temp2.columns = ['tech', 'cap'] 
        print('capacity by technology summary of table:')
        print(temp2)
        print()
        print()
        
    else:
        print('\033[1m',key,'\033[0m')
        key = pd.DataFrame.from_dict(gdx_tables[key])
        print('original table layout:')
        print(key.head(3))
        print()


 CONVOLDqctn 
original table layout:
            0     1       2       3
0  CoalOldScr   p10  recirc   101.2
1  CoalOldScr  p101  recirc  3033.0
2  CoalOldScr  p101    once  3045.0

capacity by technology summary of table:
          tech         cap
0   CoalOldScr  104268.000
1   CoalOldUns  112116.603
2     biopower    2851.201
3    coal-IGCC     443.000
4     coal-new   98321.703
5       gas-CC  193315.219
6       gas-CT  110917.601
7   geothermal    1866.500
8    lfill-gas    3104.700
9      nuclear  100719.604
10       o-g-s  127746.934


 import_store 
original table layout:
         0     1    2
0  battery  p105  2.0
1  battery  p112  2.0
2  battery  p117  2.0

capacity by technology summary of table:
           tech        cap
0       battery      6.000
1  pumped-hydro  22771.403


 others 
original table layout:
           0     1      2
0  tmpCSPOct   NaN    NaN
1         20  none    3.5
2         33  none  399.8

 nonqn 
original table layout:
      0     1     2      3
0  20

In [6]:
#CONVOLDqctn = NONRSC EXISTING CAPACITY
df = gdx_tables['CONVOLDqctn']
print(df.head())
print(df.shape)
print()

unique_0 = pd.Series(df[0].unique())
print(unique_0)
print()

unique_2 = pd.Series(df[2].unique())
print(unique_2)



            0     1       2       3
0  CoalOldScr   p10  recirc   101.2
1  CoalOldScr  p101  recirc  3033.0
2  CoalOldScr  p101    once  3045.0
3  CoalOldScr  p103  recirc  1641.0
4  CoalOldScr  p103    once   903.4
(858, 4)

0     CoalOldScr
1     CoalOldUns
2       biopower
3      coal-IGCC
4       coal-new
5         gas-CC
6         gas-CT
7     geothermal
8      lfill-gas
9        nuclear
10         o-g-s
dtype: object

0    recirc
1      once
2      pond
3       dry
4      none
dtype: object
